In [26]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time
import re
import mysql.connector
import pymysql

In [44]:
class newsScrap :
    
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"}
    
    def __init__(self, company):
        self.company = company
        self.titles = []
        self.dates = []
        self.articles = []
        self.article_urls = []
        self.press_companies = []
        
    def createNewsLinks(self):
        start_point = 1
        while True :
            #pd => 4=1일 , 2= 1달
            url = 'https://search.naver.com/search.naver?where=news&query='+ str(self.company) + '&sm=tab_opt&sort=1&photo=0&field=0&pd=4&start=' + str(start_point)
            web = requests.get(url).content
            source = BeautifulSoup(web, 'html.parser')
            
            for urls in source.find_all('a', {'class' : "info"}):
                if urls["href"].startswith("https://n.news.naver.com"):
                    self.article_urls.append(urls["href"])
#                     print(urls["href"])
            is_last_page = source.find('a',{'class':'btn_next'}).get('aria-disabled')
            if is_last_page == "true":
#                 print("last page")
                break
            else:
                time.sleep(1)
                start_point += 10
                
                    
    def newsDataScrap(self):
        for url in self.article_urls:
            try:
                web_news = requests.get(url, headers=self.headers).content
                source_news = BeautifulSoup(web_news, 'html.parser')

                title = source_news.find('h2', {'class' : 'media_end_head_headline'}).get_text()

                date = source_news.find('span', {'class' : 'media_end_head_info_datestamp_time'}).get_text()

                article = source_news.find('div', {'id' : 'newsct_article'}).get_text()
                article = article.replace("\n", "")
                article = article.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
                article = article.replace("동영상 뉴스       ", "")
                article = article.replace("동영상 뉴스", "")
                article = article.strip()

                press_company = source_news.find('em', {'class':'media_end_linked_more_point'}).get_text()
                
            
                self.titles.append(title)
                self.dates.append(date)
                self.articles.append(article)
                self.press_companies.append(press_company)
            except:
                pass
            
            
            
    def saveExcel(self):
        article_df = pd.DataFrame({'Title':self.titles, 
                                   'Date':self.dates, 
                                   'Article':self.articles, 
                                   'URL':self.article_urls, 
                                   'PressCompany':self.press_companies})
        article_df.to_excel('{}_{}.xlsx'.format(self.company, datetime.now().strftime('%y%m%d_%H%M')), index=False)
        
        
    def dbUpdater(self):
        config = {'user': 'admin_ytk',
                'password': 'tN8VnV1aBuDNvcnPWKgU',
                'host': 'database-ytk.ctkg5u6wqxe6.us-east-2.rds.amazonaws.com',
                'database': 'NLP_G3',
                'raise_on_warnings': True}
        
        cnn = pymysql.connect(host='database-ytk.ctkg5u6wqxe6.us-east-2.rds.amazonaws.com', 
                              user='admin_ytk', password='tN8VnV1aBuDNvcnPWKgU',db='NLP_G3')
        querys_data = []
        
        cur = cnn.cursor()
        
        #테이블에 맞춰 항목 추가 예정 
        for i in range(len(self.article_urls)):
            data = (i, self.titles[i], self.articles[i], self.dates[i], self.press_companies[i],self.article_urls[i])
            querys_data.append(data)
        
        query = 'INSERT INTO news_data VALUES (%s, %s, %s, %s, %s, %s)'
        #query = "INSERT INTO test (name, City) VALUES (%s, %s)"
        
        for i in querys_data:
            cur.execute(query, i)
        
        cnn.commit()
        cur.close()
        cnn.close()

In [45]:
test = newsScrap('롯데백화점')

In [46]:
test.createNewsLinks()

In [47]:
test.article_urls

['https://n.news.naver.com/mnews/article/014/0005114067?sid=101',
 'https://n.news.naver.com/mnews/article/215/0001138913?sid=101',
 'https://n.news.naver.com/mnews/article/009/0005229278?sid=103',
 'https://n.news.naver.com/mnews/article/021/0002610424?sid=101',
 'https://n.news.naver.com/mnews/article/008/0004973298?sid=101',
 'https://n.news.naver.com/mnews/article/081/0003416110?sid=102',
 'https://n.news.naver.com/mnews/article/658/0000060481?sid=101',
 'https://n.news.naver.com/mnews/article/025/0003328036?sid=101',
 'https://n.news.naver.com/mnews/article/081/0003416047?sid=101',
 'https://n.news.naver.com/mnews/article/025/0003327977?sid=101',
 'https://n.news.naver.com/mnews/article/008/0004973165?sid=101',
 'https://n.news.naver.com/mnews/article/081/0003416013?sid=102',
 'https://n.news.naver.com/mnews/article/417/0000968463?sid=101',
 'https://n.news.naver.com/mnews/article/029/0002842776?sid=101',
 'https://n.news.naver.com/mnews/article/009/0005228784?sid=103',
 'https://

In [48]:
test.newsDataScrap()

In [49]:
test.articles[0:2]

['이문•휘경뉴타운\xa0최대규모…더블\xa0역세권,\xa0생활\xa0인프라\xa0이용\xa0수월 이문 아이파크 자이 투시도_  HDC현대산업개발과 GS건설의 ‘이문 아이파크 자이’가 무순위 청약을 진행한다.   HDC현대산업개발과 GS건설은 오는 15일(금) 서울 동대문구 이문휘경뉴타운 이문3구역의 이문 아이파크 자이 일부 잔여가구를 대상으로 무순위 청약을 받는다. 이번 청약은 이문 아이파크 자이 공식 분양 홈페이지에서 오전 9시~오후 5시까지 진행한다.   누구나 청약신청이 가능하다. 현재 국내에 거주하는 만 19세 이상 또는 세대주인 미성년자(자녀양육, 형제자매부양)이면 거주지와 상관 없이 누구나 청약신청이 가능하다. 여기에 청약통장이 필요하지 않고, 실거주 의무도 없다. 다만, 이문 아이파크 자이 계약을 체결한 자 또는 당첨된 후 계약을 체결하지 않은 자, 부적격 당첨자로 그 제한기간 중에 있는 자 등은 청약신청이 불가하다.   이번 무순위 청약 당첨자 발표는 청약당일인 15일(금)이며, 계약은 19일(화) 견본주택에서 진행한다.   한편, ‘이문 아이파크 자이’는 지하 6층~지상 최고 41층 25개동 총 4321가구 규모로 이중 전용면적 20~102㎡ 총 1467가구가 일반에 공급됐다.   지하철을 걸어서 이용할 수 있는 데다가 자연환경 또한 누릴 수 있다는 설명이다. 서울지하철 1호선 외대앞역 초역세권인데다 서울 지하철 1호선 신이문역도 걸어서 이용할 수 있다. 천장산과 중랑천 등을 걸어서 이용할 수 있어 여가활동은 물론 쾌적한 주거생활이 가능하다. 특히 3단지는 천장산을 품고 있는 도심 숲세권 단지로 쾌적하고 여유로운 주거 라이프를 누릴 수 있는 것이 특징이다.   교육환경도 눈길을 끈다. 이문초, 석관중, 석관고, 경희중, 경희고 등 각급학교가 도보권에 위치해 있어 자녀들의 안전한 통학이 가능하고, 인근으로 한국외대, 경희대, 한예종, 카이스트 서울캠퍼스 등이 밀집해 있다. 또한 롯데백화점 청량리점, 코스트코 상봉점, 홈플러스 상봉점, 이마트 묵

In [50]:
test.dbUpdater()

IntegrityError: (1452, 'Cannot add or update a child row: a foreign key constraint fails (`NLP_G3`.`news_data`, CONSTRAINT `fk_enter_id` FOREIGN KEY (`enter_id`) REFERENCES `enterprise_data` (`enter_id`))')

In [39]:
def divideArticle(artList):
    divideArt = []
    for art in artList:
        artTemp = art.split('.')
        divideArt.append(artTemp)
    return divideArt

In [40]:
ditest = divideArticle(test.articles[0:2])

In [51]:
config = {'user': 'admin_ytk',
            'password': 'tN8VnV1aBuDNvcnPWKgU',
            'host': 'database-ytk.ctkg5u6wqxe6.us-east-2.rds.amazonaws.com',
            'database': 'NLP_G3',
            'raise_on_warnings': True}
cnn = pymysql.connect(host='database-ytk.ctkg5u6wqxe6.us-east-2.rds.amazonaws.com', user='admin_ytk', password='tN8VnV1aBuDNvcnPWKgU',db='NLP_G3')        
cur = cnn.cursor()
# connection = mysql.connector.connect(**config)
        
# cursor = connection.cursor()

# query = 'SELECT * FROM test'
query2 = "INSERT INTO test (name, City) VALUES (%s, %s)"
data = ("fqwfgbrw", "regrh")
cur.execute(query2, data)
cnn.commit()

# result = cursor.fetchall()
# for row in result:
#      print(row)
    
cursor.close()
connection.close()

In [65]:
config = {'user': 'admin_ytk',
            'password': 'tN8VnV1aBuDNvcnPWKgU',
            'host': 'database-ytk.ctkg5u6wqxe6.us-east-2.rds.amazonaws.com',
            'database': 'NLP_G3',
            'raise_on_warnings': True}
cnn = pymysql.connect(host='database-ytk.ctkg5u6wqxe6.us-east-2.rds.amazonaws.com', user='admin_ytk', password='tN8VnV1aBuDNvcnPWKgU',db='NLP_G3')        
cur = cnn.cursor()

query = 'SELECT * FROM news_data'
cur.execute(query)


result = cur.fetchall()
for row in result:
     print(row)
cur.close()
cnn.close()

(1, '철도시설 유지보수 코레일 독점깨지나', '\'독점보장 단서조항 삭제\' 철산법 개정안 장기 표류에정부, 시행령 개정 만지작···"민영화 아냐, 공공기관 한정"[서울경제] 정부가 철도시설 유지보수 업무의 한국철도공사(코레일) 독점 구조 해소 방안을 추진한다. 코레일 내에는 ‘안전부사장’직을 신설해 유지보수의 책임성을 강화하고 관제의 독립성을 강화한다.국토교통부는 지난해 12월 조응천 더불어민주당 의원이 대표발의한 철도산업발전기본법(철산법) 개정안이 국토교통위원회(교통법안심사소위원회)에서 조속히 통과되도록 철도노조와 국회를 설득할 계획이라고 14일 밝혔다. 철산법 38조는 철도관제, 철도시설 유지보수 등 업무를 대통령령에서 위탁할 수 있다고 규정하면서 ‘단, 철도시설 유지보수 시행 업무는 철도공사에 위탁한다’고 단서를 달아놨다. 이 때문에 코레일은 현재 철도시설 유지보수 업무를 사실상 독점하고 있다.문제는 2003년 철산법 제정 당시와 달리 철도운영기관이 다양화하면서 국가철도 중 진접선 등과 같이 코레일이 운영하지 않는 노선까지 코레일이 유지보수를 수행하는 비효율이 발생하고 있다. 운영기관과 유지보수기관을 일치시키는 제도적 틀을 마련하는 게 시급한 이유다.조 의원이 발의한 철산법 개정안은 코레일의 독점적인 유지보수를 보장한 단서조항을 삭제하는 것이 골자다. 하지만 철도노조와 일부 야당 의원이 철산법 개정으로 유지보수 등이 민영화될 수 있다며 강하게 반대하면서 1년간 철산법 개정안은 국회에 잠들어 있는 상태다. 이에 국토부는 코레일 운영 구간은 코레일이, 나머지 구간은 해당 운영사가 유지보수를 수행하도록 시행령에 규정하는 일종의 절충안을 국회에 제안했다. 이를 적용할 경우 2016년 개통된 수서평택고속선의 유지보수 업무는 기존 코레일에서 운영기관인 SR로 이관돼야 한다.국토부는 민영화 우려를 불식시키기 위한 장치도 마련 중이다. 박재순 국토부 철도안전정책관은 “철산법 개정이 ‘민영화’는 절대 아니다”라며 “유지보수를 민간에 개방하는 것이 아니라, 서울교통공

In [34]:
qd = []

def dbUpdater():
    config = {'user': 'admin_ytk',
            'password': 'tN8VnV1aBuDNvcnPWKgU',
            'host': 'database-ytk.ctkg5u6wqxe6.us-east-2.rds.amazonaws.com',
            'database': 'NLP_G3',
            'raise_on_warnings': True}
    
    cnn = pymysql.connect(host='database-ytk.ctkg5u6wqxe6.us-east-2.rds.amazonaws.com', 
                              user='admin_ytk', password='tN8VnV1aBuDNvcnPWKgU',db='NLP_G3')        
        
    cur = cnn.cursor()
        
        #테이블에 맞춰 항목 추가 예정 
#     for i in range(len(self.article_urls)):
#         data = (self.titles[i], self.dates[i])
#         querys_data.append(data)
        
        #query = 'INSERT INTO news_data VALUES (%s, %s)'
    
    qd.append(('wr','wqdqwd'))
    qd.append(('uyk','htyh'))
    qd.append(('gewg','ew'))
    
    
    
    
    query = "INSERT INTO test (name, City) VALUES (%s, %s)"
        
    for i in qd:
        cur.execute(query, i)
        
    cnn.commit()

In [62]:
class newsScrap :
    
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"}
    
    def __init__(self, company):
        self.company = company
        self.titles = []
        self.dates = []
        self.articles = []
        self.article_urls = []
        self.press_companies = []
        self.url_full = []
        self.tests = []
        
    def createNewsLinks(self):
        start_point = 1
        while True :
            #pd => 4=1일 , 2= 1달
            url = 'https://search.naver.com/search.naver?where=news&query='+ str(self.company) + '&sm=tab_opt&sort=1&photo=0&field=0&pd=4&start=' + str(start_point)
            web = requests.get(url).content
            source = BeautifulSoup(web, 'html.parser')
            
            for urls in source.find_all('a', {'class' : "info"}):
                if urls["href"].startswith("https://n.news.naver.com"):
                    self.article_urls.append(urls["href"])
                    # print(urls["href"])
            is_last_page = source.find('a',{'class':'btn_next'}).get('aria-disabled')
            if is_last_page == "true":
                # print("last page")
                break
            else:
                time.sleep(1)
                start_point += 10
                
                    
    def newsDataScrap(self):
        test = 0
        for url in self.article_urls:
            try:
                web_news = requests.get(url, headers=self.headers).content
                source_news = BeautifulSoup(web_news, 'html.parser')

                title = source_news.find('h2', {'class' : 'media_end_head_headline'}).get_text()

                date = source_news.find('span', {'class' : 'media_end_head_info_datestamp_time'}).get_text()

                article = source_news.find('div', {'id' : 'newsct_article'}).get_text()
                article = article.replace("\n", "")
                article = article.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
                article = article.replace("동영상 뉴스       ", "")
                article = article.replace("동영상 뉴스", "")
                article = article.strip()

                press_company = source_news.find('em', {'class':'media_end_linked_more_point'}).get_text()
                
                
                test += 1
                
                self.tests.append(test)
                self.titles.append(title)
                self.dates.append(date)
                self.articles.append(article)
                self.press_companies.append(press_company)
                self.url_full.append(url)
            except:
                pass
            
            
            
    def saveExcel(self):
        article_df = pd.DataFrame({'Title':self.titles, 
                                   'Date':self.dates, 
                                   'Article':self.articles, 
                                   'URL':self.url_full, 
                                   'PressCompany':self.press_companies})
        article_df.to_excel('{}_{}.xlsx'.format(self.company, datetime.now().strftime('%y%m%d_%H%M')), index=False)
        
        
    def dbUpdater(self):
        config = {'user': 'admin_ytk',
                'password': 'tN8VnV1aBuDNvcnPWKgU',
                'host': 'database-ytk.ctkg5u6wqxe6.us-east-2.rds.amazonaws.com',
                'database': 'NLP_G3',
                'raise_on_warnings': True}
        
        cnn = pymysql.connect(host='database-ytk.ctkg5u6wqxe6.us-east-2.rds.amazonaws.com', 
                              user='admin_ytk', password='tN8VnV1aBuDNvcnPWKgU',db='NLP_G3')
        querys_data = []
        
        cur = cnn.cursor()
        
        #테이블에 맞춰 항목 추가 예정 
        for i in range(5):
            data = (self.tests[i], self.titles[i], self.articles[i], self.dates[i], 'ER01', self.article_urls[i])
            querys_data.append(data)
        
        query = 'INSERT INTO news_data VALUES (%s, %s, %s, %s, %s, %s)'
        #query = "INSERT INTO test (name, City) VALUES (%s, %s)"
        
        for i in querys_data:
            cur.execute(query, i)
        
        cnn.commit()
        cur.close()
        cnn.close()

In [63]:
test = newsScrap('LH')
test.createNewsLinks()
print(test.article_urls)
print(len(test.article_urls))
test.newsDataScrap()
print(test.titles)
print(len(test.titles))
print(len(test.url_full))

['https://n.news.naver.com/mnews/article/011/0004274412?sid=101', 'https://n.news.naver.com/mnews/article/081/0003416608?sid=102', 'https://n.news.naver.com/mnews/article/374/0000362795?sid=101', 'https://n.news.naver.com/mnews/article/003/0012267265?sid=100', 'https://n.news.naver.com/mnews/article/081/0003416605?sid=102', 'https://n.news.naver.com/mnews/article/056/0011622339?sid=102', 'https://n.news.naver.com/mnews/article/016/0002239001?sid=101', 'https://n.news.naver.com/mnews/article/003/0012267161?sid=102', 'https://n.news.naver.com/mnews/article/079/0003843509?sid=102', 'https://n.news.naver.com/mnews/article/021/0002610818?sid=102', 'https://n.news.naver.com/mnews/article/654/0000060354?sid=102', 'https://n.news.naver.com/mnews/article/015/0004925388?sid=101', 'https://n.news.naver.com/mnews/article/374/0000362778?sid=101', 'https://n.news.naver.com/mnews/article/001/0014390481?sid=101', 'https://n.news.naver.com/mnews/article/047/0002415832?sid=102', 'https://n.news.naver.co

In [57]:
test.saveExcel()

In [64]:
test.dbUpdater()